**CLASSIFICATION**


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

In [ ]:
len(os.listdir("/content/DTA/email"))

55

In [ ]:
len(os.listdir("/content/DTA/resume"))

55

In [ ]:
len(os.listdir("/content/DTA/scientific"))

55

In [ ]:
len(os.listdir("/content/path_to_output_folder/train/email"))

33

In [ ]:
len(os.listdir("/content/DTA/scientific"))

In [ ]:
len(os.listdir("/content/DTA/scientific"))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    os.path.join(output_base_dir, 'train'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Validation data generator
validation_generator = val_test_datagen.flow_from_directory(
    os.path.join(output_base_dir, 'val'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Test data generator
test_generator = val_test_datagen.flow_from_directory(
    os.path.join(output_base_dir, 'test'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 99 images belonging to 3 classes.
Found 33 images belonging to 3 classes.
Found 33 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Load pre-trained VGG16 model + higher level layers
base_model = VGG16(input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

# Build the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes: resume, email, scientific publication
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


58889256/58889256 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Total params: 21138243 (80.64 MB)
Trainable params: 6423555 (24.50 MB)
Non-trainable params: 14714688 (56.13 MB)
__________

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,  # Adjust the number of epochs as needed
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

Epoch 1/10
3/3 [==============================] - 68s 31s/step - loss: 4.7130 - accuracy: 0.3134 - val_loss: 1.2266 - val_accuracy: 0.6250
Epoch 2/10
3/3 [==============================] - 66s 32s/step - loss: 3.5696 - accuracy: 0.3582 - val_loss: 1.1114 - val_accuracy: 0.5000
Epoch 3/10
3/3 [==============================] - 84s 31s/step - loss: 1.6360 - accuracy: 0.5522 - val_loss: 1.5634 - val_accuracy: 0.6562
Epoch 4/10
3/3 [==============================] - 84s 40s/step - loss: 2.2509 - accuracy: 0.5821 - val_loss: 0.3082 - val_accuracy: 0.8125
Epoch 5/10
3/3 [==============================] - 64s 21s/step - loss: 2.3272 - accuracy: 0.5373 - val_loss: 0.2434 - val_accuracy: 0.9375
Epoch 6/10
3/3 [==============================] - 64s 21s/step - loss: 1.8703 - accuracy: 0.6119 - val_loss: 0.1604 - val_accuracy: 0.9688
Epoch 7/10
3/3 [==============================] - 66s 22s/step - loss: 1.5321 - accuracy: 0.7015 - val_loss: 0.1409 - val_accuracy: 0.9375
Epoch 8/10
3/3 [===========

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy:.2f}")

2/2 [==============================] - 20s 707ms/step - loss: 1.5608 - accuracy: 0.7879
Test accuracy: 0.79


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
from sklearn.metrics import classification_report

# Predict on test data
test_generator.reset()  # Reset the generator to start from the beginning
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get the true labels from the generator
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Print out a classification report
print(classification_report(true_classes, predicted_classes, target_names=class_labels))


2/2 [==============================] - 21s 530ms/step
              precision    recall  f1-score   support

       email       1.00      0.64      0.78        11
      resume       0.65      1.00      0.79        11
  scientific       0.89      0.73      0.80        11

    accuracy                           0.79        33
   macro avg       0.85      0.79      0.79        33
weighted avg       0.85      0.79      0.79        33



In [ ]:
true_labels = test_generator.classes

# Predict the classes for the test set
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Calculate the number of correctly classified samples
correct_predictions = np.sum(predicted_classes == true_labels)


2/2 [==============================] - 28s 548ms/step


In [ ]:
print(f'Total test samples: {len(true_labels)}')
print(f'Correctly classified test samples: {correct_predictions}')


Total test samples: 33
Correctly classified test samples: 26
